In [1]:
from datetime import datetime
import settings.config as cfg


print(cfg.individual_rs_strategy)
preprocessed_dataset_folder = cfg.preprocessed_dataset_folder
individual_rs_strategy = cfg.individual_rs_strategy
aggregation_strategies = cfg.aggregation_strategies
recommendations_number = cfg.recommendations_number
individual_rs_validation_folds_k = cfg.individual_rs_validation_folds_k
group_rs_evaluation_folds_k = cfg.group_rs_evaluation_folds_k
evaluation_strategy = cfg.evaluation_strategy
metrics = cfg.metrics

LENSKIT_CF_USER


In [2]:
import pandas as pd
ratings_df = pd.read_csv(preprocessed_dataset_folder+"/ratings.csv")

import pickle

group_composition = pickle.load(open(preprocessed_dataset_folder+"/group_composition.pkl", "rb"))
display(group_composition)

{0: {'group_size': 2,
  'group_similarity': 'random',
  'group_members': [4805, 5428]},
 1: {'group_size': 2,
  'group_similarity': 'random',
  'group_members': [5251, 146]},
 2: {'group_size': 2,
  'group_similarity': 'random',
  'group_members': [3916, 4539]},
 3: {'group_size': 2,
  'group_similarity': 'random',
  'group_members': [2059, 5558]},
 4: {'group_size': 2,
  'group_similarity': 'random',
  'group_members': [1789, 463]},
 5: {'group_size': 2,
  'group_similarity': 'random',
  'group_members': [3234, 4068]},
 6: {'group_size': 2,
  'group_similarity': 'random',
  'group_members': [5216, 4855]},
 7: {'group_size': 2,
  'group_similarity': 'random',
  'group_members': [339, 5736]},
 8: {'group_size': 2,
  'group_similarity': 'random',
  'group_members': [153, 4515]},
 9: {'group_size': 2,
  'group_similarity': 'random',
  'group_members': [3450, 2157]},
 10: {'group_size': 3,
  'group_similarity': 'random',
  'group_members': [707, 2380, 379]},
 11: {'group_size': 3,
  'group

In [3]:
# Auxiliary functions

# Train individual recommender system and predict ratings
def train_individual_rs_and_get_predictions(training_df, test_df):
    if cfg.individual_rs_strategy == "LENSKIT_CF_USER":
        print(cfg.individual_rs_strategy)
        return train_lenskit_cf_user_rs_and_get_predictions(training_df, test_df)
    return None
    
from lenskit.algorithms import Recommender
from lenskit.algorithms.user_knn import UserUser

# Train lenskit CF user-user individual recommender system and predict ratings
def train_lenskit_cf_user_rs_and_get_predictions(training_df, test_df):
    if cfg.individual_rs_validation_folds_k <=0:
        print("training")
        # Basic implementation: no hyperparameters validation
        user_user = UserUser(15, min_nbrs=3)  # Minimum (3) and maximum (15) number of neighbors to consider
        recsys = Recommender.adapt(user_user)
        recsys.fit(training_df)
        
        print("evaluating predictions")
        # Evaluating predictions 
        test_df['predicted_rating'] = recsys.predict(test_df)
        print("Done!")
        return test_df
    return None    

In [4]:
import numpy as np

# Aggregation strategies

from abc import ABC, abstractmethod

class AggregationStrategy(ABC):
    
    @staticmethod
    def getAggregator(strategy):            
        if strategy=="ADD":
            return AdditiveAggregator()
        elif strategy=="LMS":
            return LeastMiseryAggregator()
        elif strategy=="BASE":
            return BaselinesAggregator()
        return None
    
#     @abstractmethod
#     def generate_group_recommendations_forall_groups(self, test_df, group_composition, recommendations_number):
#         pass
    
    @abstractmethod
    def generate_group_recommendations_for_group(self, group_ratings, recommendations_number):
        pass
    

class AdditiveAggregator(AggregationStrategy):
#     def generate_group_recommendations_forall_groups(self, test_df, group_composition, recommendations_number):
#         group_recommendations = dict()
#         for group_id in group_composition:
#             # extract group info
#             group = group_composition[group_id]
#             group_size = group['group_size']
#             group_similarity = group['group_similarity']
#             group_members = group['group_members']
            
#             # filter ratings for the group members
#             group_ratings = test_df.loc[test_df['user'].isin(group_members)]
            
#             # aggregate using additive strategy
#             aggregated_df = group_ratings.groupby('item').sum()
#             aggregated_df = aggregated_df.sort_values(by="predicted_rating", ascending=False).reset_index()[['item', 'predicted_rating']]
#             recommendation_list = list(aggregated_df.head(recommendations_number)['item'])
            
#             group_recommendations[group_id] = recommendation_list
            
#         return group_recommendations
    
    def generate_group_recommendations_for_group(self, group_ratings, recommendations_number):
        aggregated_df = group_ratings.groupby('item').sum()
        aggregated_df = aggregated_df.sort_values(by="predicted_rating", ascending=False).reset_index()[['item', 'predicted_rating']]
        recommendation_list = list(aggregated_df.head(recommendations_number)['item'])
        return {"ADD" : recommendation_list}
    
class LeastMiseryAggregator(AggregationStrategy):
#     def generate_group_recommendations_forall_groups(self, test_df, group_composition, recommendations_number):
#         group_recommendations = dict()
#         for group_id in group_composition:
#             # extract group info
#             group = group_composition[group_id]
#             group_size = group['group_size']
#             group_similarity = group['group_similarity']
#             group_members = group['group_members']
            
#             # filter ratings for the group members
#             group_ratings = test_df.loc[test_df['user'].isin(group_members)]
            
#             # aggregate using least misery strategy
#             aggregated_df = group_ratings.groupby('item').min()
#             aggregated_df = aggregated_df.sort_values(by="predicted_rating", ascending=False).reset_index()[['item', 'predicted_rating']]
#             recommendation_list = list(aggregated_df.head(recommendations_number)['item'])
            
#             group_recommendations[group_id] = recommendation_list
            
#         return group_recommendations
    
    def generate_group_recommendations_for_group(self, group_ratings, recommendations_number):
        # aggregate using least misery strategy
        aggregated_df = group_ratings.groupby('item').min()
        aggregated_df = aggregated_df.sort_values(by="predicted_rating", ascending=False).reset_index()[['item', 'predicted_rating']]
        recommendation_list = list(aggregated_df.head(recommendations_number)['item'])
        return {"LMS" : recommendation_list}

class BaselinesAggregator(AggregationStrategy):
#     def generate_group_recommendations_forall_groups(self, test_df, group_composition, recommendations_number):
#         return None
    
    def generate_group_recommendations_for_group(self, group_ratings, recommendations_number):
        # aggregate using least misery strategy
        aggregated_df = group_ratings.groupby('item').agg({"predicted_rating": [np.sum, np.prod,np.min,np.max]})
        aggregated_df = aggregated_df["predicted_rating"].reset_index()
        # additive
        
        add_df = aggregated_df.sort_values(by="sum", ascending=False).reset_index()[['item', 'sum']]
        add_recommendation_list = list(add_df.head(recommendations_number)['item'])
        # multiplicative
        mul_df = aggregated_df.sort_values(by="prod", ascending=False).reset_index()[['item', 'prod']]
        mul_recommendation_list = list(mul_df.head(recommendations_number)['item'])
        # least misery
        lms_df = aggregated_df.sort_values(by="amin", ascending=False).reset_index()[['item', 'amin']]
        lms_recommendation_list = list(lms_df.head(recommendations_number)['item'])
        # most pleasure
        mpl_df = aggregated_df.sort_values(by="amax", ascending=False).reset_index()[['item', 'amax']]
        mpl_recommendation_list = list(mpl_df.head(recommendations_number)['item'])
        return {
            "ADD" : add_recommendation_list, 
            "MUL" : mul_recommendation_list, 
            "LMS" : lms_recommendation_list, 
            "MPL" : mpl_recommendation_list
        }

In [5]:
# Evaluating recommendations for all the aggregation strategies

# def generate_group_recommendations_forall_aggr_strat(test_df, group_composition, recommendations_number):
#     group_recommendations = dict()
#     for aggregation_strategy in cfg.aggregation_strategies:
#         print(datetime.now(), aggregation_strategy)
#         agg = AggregationStrategy.getAggregator(aggregation_strategy)
#         group_recommendations[aggregation_strategy] = agg.generate_group_recommendations_forall_groups(test_df, group_composition, recommendations_number)
        
#     return group_recommendations

def generate_group_recommendations_forall_groups(test_df, group_composition, recommendations_number):
    group_recommendations = dict()
    for group_id in group_composition:
        
#         print(datetime.now(), group_id)
        
        # extract group info
        group = group_composition[group_id]
        group_size = group['group_size']
        group_similarity = group['group_similarity']
        group_members = group['group_members']
            
        # filter ratings for the group members
        group_ratings = test_df.loc[test_df['user'].isin(group_members)]
        
        group_rec = dict()
        for aggregation_strategy in cfg.aggregation_strategies:
#             print(datetime.now(), aggregation_strategy)
            agg = AggregationStrategy.getAggregator(aggregation_strategy)
            group_rec = {**group_rec, **agg.generate_group_recommendations_for_group(group_ratings, recommendations_number)}
        
        
        group_recommendations[group_id] = group_rec
        
    return group_recommendations

In [12]:
import numpy as np

# Evaluation Metrics strategies

from abc import ABC, abstractmethod

class MetricEvaluator(ABC):
    
    @staticmethod
    def getMetricEvaluator(metric):            
        if metric=="NDCG":
            return NDCGEvaluator()
        elif metric=="BASE":
            return BaselinesEvaluators()
        return None
    
    @abstractmethod
    def evaluateGroupRecommendation(self, group_ground_truth, group_recommendation, group_members):
        pass
    

class NDCGEvaluator(MetricEvaluator):
    
    def evaluateUserNDCG(self, user_ground_truth, group_recommendation):
        dcg = 0
#         display(user_ground_truth)
#         display(group_recommendation)
        for k, item in zip(range(len(group_recommendation)), group_recommendation):
            dcg = dcg + ((1 if item in user_ground_truth else 0) / np.log2(k+2))
        
        idcg = 0
        # what if intersection is empty?
        for k in range(max(len(user_ground_truth),len(set(group_recommendation)))):
            idcg = idcg + (1 / np.log2(k+2))
            
        ndcg = dcg / idcg
        print(user_ground_truth, group_recommendation, dcg, idcg, ndcg)

        return ndcg
        
    def evaluateGroupRecommendation(self, group_ground_truth, group_recommendation, group_members):
        ndcg_list = list()
        for user in group_members:
            # evaluate 
            user_ground_truth = ground_truth.loc[ground_truth['user']==user]
            user_ground_truth = set(user_ground_truth.loc[user_ground_truth['rating']>=4]['item'])
            ndcg_user = self.evaluateUserNDCG(user_ground_truth, group_recommendation)
            ndcg_list.append(ndcg_user)
        
        return {"NDCG" : {
            "mean" : np.mean(ndcg_list),
            "min" : np.amin(ndcg_list)
        }}
    

class BaselinesEvaluators(MetricEvaluator):
    def evaluateGroupRecommendation(self, group_ground_truth, group_recommendation, group_members):
        return None
    

In [13]:
def evaluate_group_recommendations_forall_groups(ground_truth, group_recommendations, group_composition):
    group_evaluations = dict()
    for group_id in group_composition:
        
        print(datetime.now(), group_id)
        
        # extract group info
        group = group_composition[group_id]
        group_size = group['group_size']
        group_similarity = group['group_similarity']
        group_members = group['group_members']
        group_rec = group_recommendations[group_id]
            
        # filter ratings in ground_truth for the group members
        group_ground_truth = ground_truth.loc[ground_truth['user'].isin(group_members)]
        
        group_rec_eval = dict()
        for aggregation_strategy in group_rec:
            agg_group_rec = group_rec[aggregation_strategy]
            agg_group_rec_eval = dict()
            for metric in cfg.metrics:
    #             print(datetime.now(), aggregation_strategy)
                metric_evaluator = MetricEvaluator.getMetricEvaluator(metric)
                agg_group_rec_eval = {**agg_group_rec_eval, **metric_evaluator.evaluateGroupRecommendation(group_ground_truth, agg_group_rec, group_members)}

            group_rec_eval[aggregation_strategy] = agg_group_rec_eval

        
        group_evaluations[group_id] = group_rec_eval
        
    return group_evaluations

In [14]:
# General pipeline

# creating train-test folds
# split stratified on the users 

from sklearn.model_selection import StratifiedKFold
import itertools

print(datetime.now(), "Creating folds")
# skf = StratifiedKFold(n_splits=group_rs_evaluation_folds_k, random_state=None, shuffle=True)
skf = StratifiedKFold(n_splits=group_rs_evaluation_folds_k, random_state=None, shuffle=True)

print(datetime.now(), "Folds created!")
for train_index, test_index in skf.split(ratings_df, ratings_df['user']):
    print(">>> Start processing fold: Train", len(train_index), "Test:", len(test_index))
    
    # split train and test df
    train_df = ratings_df.iloc[train_index]
    test_df = ratings_df.iloc[test_index]
    
#     print(datetime.now(), "Train and test")
#     display(train_df)
#     display(test_df)
    
    # create test_complete_df with all the possible user-items pairs in the test_df
    user_set = set(test_df['user'].values)
    item_set = set(test_df['item'].values)
    test_pred_df = pd.DataFrame(list(itertools.product(user_set, item_set)), columns=['user', 'item'])
    
#     print(datetime.now(), "Extended test df")
#     display(test_pred_df)
    
    print(datetime.now(), "Train individual RS and get predictions")
    # train individual rs and get predictions
    test_pred_df = train_individual_rs_and_get_predictions(train_df, test_pred_df)

    print(datetime.now(), "Generate GRS for all the aggregation strategies and all the groups")
    # - generate the recommendations for all the aggregation strategies and all the groups
    group_recommendations = generate_group_recommendations_forall_groups(test_pred_df, group_composition, cfg.recommendations_number)
    
    # - evaluate the recommendations
    if cfg.evaluation_strategy == "COUPLED":
        ground_truth = test_df
    else:
        ground_truth = test_pred_df.rename(columns={"predicted_rating": "rating"}, errors="raise")
        
    fold_group_evaluations = evaluate_group_recommendations_forall_groups(ground_truth, group_recommendations, group_composition)
    display(fold_group_evaluations)
    
# Finally, we merge the results for all the folds

2022-09-09 13:23:39.201710 Creating folds
2022-09-09 13:23:39.203677 Folds created!
>>> Start processing fold: Train 753780 Test: 188445
2022-09-09 13:24:24.863917 Train individual RS and get predictions
LENSKIT_CF_USER
training
evaluating predictions
Done!
2022-09-09 13:31:08.755800 Generate GRS for all the aggregation strategies and all the groups
2022-09-09 13:31:30.864302 0
{3793, 3745, 318, 3751} [318, 2858, 1207, 858, 50, 2762, 527, 1262, 260, 750] 1.0 4.543559338088346 0.22009176629808017
{1794, 4, 3082, 2826, 527, 2320, 1682, 22, 151, 534, 25, 3101, 1660, 1569, 1573, 2599, 1193, 3247, 1968, 1721, 1466, 954, 2875, 1339, 3135, 2112, 1221, 1225, 2762, 2763, 1097, 2125, 590, 207, 1358, 1366, 1240, 2269, 991, 2145, 3555, 358, 1518, 1271, 1784, 2297, 507, 380, 509, 1399} [318, 2858, 1207, 858, 50, 2762, 527, 1262, 260, 750] 0.6895405204413555 12.89773270186745 0.053462150005753925
{3793, 3745, 318, 3751} [318, 2858, 1207, 858, 50, 2762, 527, 1262, 750, 1252] 1.0 4.543559338088346 0.2

{2028, 2997} [318, 50, 527, 2762, 922, 2329, 3469, 954, 2019, 1223] 0.0 4.543559338088346 0.0
{480, 3740, 3175, 457, 1196, 592, 1584, 1682, 1396, 1270, 700} [318, 50, 527, 2762, 922, 2329, 3469, 954, 2019, 1223] 0.0 4.8225022837394755 0.0
{2028, 2997} [2858, 2028, 318, 1197, 50, 1203, 1196, 1198, 2959, 593] 0.6309297535714575 4.543559338088346 0.13886244387355454
{480, 3740, 3175, 457, 1196, 592, 1584, 1682, 1396, 1270, 700} [2858, 2028, 318, 1197, 50, 1203, 1196, 1198, 2959, 593] 0.3333333333333333 4.8225022837394755 0.06912040963822193
2022-09-09 13:31:31.264439 7
{1952, 3105, 3201, 509, 1124, 3461, 971, 3467, 3468, 2764, 1362, 3451, 1172, 25, 58, 923, 1949} [1178, 1207, 1361, 318, 50, 923, 1203, 2019, 912, 858] 0.3562071871080222 6.345811006700162 0.0561326498270945
{1217, 1249, 3265, 1284, 899, 1287, 1961, 1259, 1387, 110, 527, 593, 1238, 1278, 892, 1150} [1178, 1207, 1361, 318, 50, 923, 1203, 2019, 912, 858] 0.0 6.10599854013203 0.0
{1952, 3105, 3201, 509, 1124, 3461, 971, 3467, 3

{2706, 2826} [50, 1148, 858, 3030, 527, 318, 2762, 1234, 913, 356] 0.0 4.543559338088346 0.0
{2336, 3168, 34, 1924, 2757, 1956, 1407, 2858, 3179, 2762, 2125, 1244, 3476, 2997, 3095, 892, 735} [50, 1148, 858, 3030, 527, 318, 2762, 1234, 913, 356] 0.3333333333333333 6.345811006700162 0.05252809026007655
{260, 778, 3210, 1552, 1297, 785, 2194, 1301, 2841, 3740, 1196, 1580, 2094, 1198, 50, 2997, 2490, 1466, 2363, 3386, 2114, 3267, 2883, 2371, 3526, 1091, 1610, 590, 2000, 593, 1748, 2518, 1370, 1374, 480, 2402, 610, 2918, 1127, 2028, 1517, 3697, 3703, 3576, 1663} [50, 1148, 858, 3030, 527, 318, 2762, 1234, 913, 356] 1.0 12.007070295381267 0.0832842629716816
{2706, 2826} [2858, 2997, 1198, 1196, 260, 50, 2571, 1207, 318, 2396] 0.0 4.543559338088346 0.0
{2336, 3168, 34, 1924, 2757, 1956, 1407, 2858, 3179, 2762, 2125, 1244, 3476, 2997, 3095, 892, 735} [2858, 2997, 1198, 1196, 260, 50, 2571, 1207, 318, 2396] 1.6309297535714575 6.345811006700162 0.2570088759103378
{260, 778, 3210, 1552, 1297, 78

{2, 2052, 5, 2572, 1036, 524, 3087, 2072, 2596, 2087, 3624, 2094, 2606, 3120, 47, 587, 2124, 1615, 1616, 2640, 595, 3156, 2133, 112, 2169, 2683, 3708, 1665, 1676, 2701, 671, 2719, 673, 1702, 1198, 3248, 1200, 1210, 2747, 1214, 3264, 1220, 2253, 737, 239, 2291, 3827, 248, 1275, 3328, 258, 784, 788, 2329, 3354, 798, 1831, 296, 818, 2375, 1356, 2389, 344, 1882, 1370, 349, 1387, 364, 367, 1393, 377, 379, 1920, 1921, 3477, 415, 419, 2987, 1967, 1968, 435, 2997, 3004, 1479, 2001, 2011, 1500, 3039, 2529, 1515, 1517, 2542, 500} [318, 50, 2858, 2329, 527, 2571, 2324, 3030, 3578, 1178] 0.43067655807339306 19.880354329226844 0.02166342465235841
{450, 2724, 552, 3499, 588, 1645, 473, 879, 3120, 141, 2514, 788, 1367, 1625, 1597, 1918, 1855} [318, 50, 2858, 2329, 527, 2571, 2324, 3030, 3578, 1178] 0.0 6.345811006700162 0.0
{2502, 3175, 2504, 2329, 2572, 2700, 2712, 2959, 16, 1617, 3186, 2291, 1680, 1393, 22, 2709, 2710, 1277, 2683, 1213} [318, 1178, 2329, 913, 3578, 527, 2019, 3030, 2324, 2571] 0.5 

{608, 1304, 1203, 527} [1262, 858, 1193, 318, 50, 593, 1219, 527, 2329, 2028] 0.31546487678572877 4.543559338088346 0.06943122193677727
{1625, 1127, 3758, 207} [1262, 858, 1193, 318, 50, 593, 1219, 527, 2329, 2028] 0.0 4.543559338088346 0.0
{1729, 2348, 2028, 110, 3255, 858} [1262, 858, 1193, 318, 50, 593, 1219, 527, 2329, 2028] 0.9199945798893454 4.543559338088346 0.20248323207250624
{3072, 3330, 1284, 903, 905, 1294, 912, 913, 3088, 1041, 3476, 1942, 3095, 3608, 1689, 2202, 2328, 2076, 3101, 1694, 159, 2080, 3105, 1953, 1699, 932, 2723, 541, 1962, 3499, 2348, 1965, 1968, 2097, 1200, 2612, 2359, 1848, 3260, 1340, 2750, 1982, 2496, 1089, 2243, 1347, 1219, 2118, 3271, 3398, 708, 1358, 2513, 1748, 2133, 471, 2648, 380, 3418, 1883, 3546, 3037, 1885, 3424, 994, 1252, 2407, 999, 2664, 1258, 2921, 2542, 1904, 3699, 1784, 1276, 25} [1262, 858, 1193, 318, 50, 593, 1219, 527, 2329, 2028] 0.3333333333333333 17.39265712989499 0.01916517590405382
2022-09-09 13:31:32.400481 22
{2947, 2692, 1287, 34

...

{1, 3114, 3082, 1197, 2396, 924} [3089, 904, 912, 527, 3030, 1267, 953, 1204, 1178, 1148] 0.0 4.543559338088346 0.0
{930, 3334, 3435, 942, 2935, 912, 3095, 2203} [3089, 904, 912, 527, 3030, 1267, 953, 1204, 1178, 1148] 0.5 4.543559338088346 0.11004588314904008
{908, 3471, 2194, 541, 3114, 1199, 1207, 1208, 955, 1084, 318, 1219, 581, 2247, 589, 2395, 3676, 3037, 2013, 2788, 3685, 750, 1265, 2932, 1269} [3089, 904, 3030, 912, 1267, 1178, 1204, 527, 2019, 953] 0.0 8.131765560174182 0.0
{2690, 1411, 900, 260, 1801, 2571, 1931, 919, 26, 3483, 1566, 2718, 2081, 39, 3624, 2985, 3751, 2092, 3635, 3638, 3639, 1210, 1722, 316, 318, 2629, 454, 2376, 1610, 1356, 592, 2641, 1876, 3668, 2393, 1372, 350, 1375, 1376, 1380, 1010, 1013, 3448, 377} [3089, 904, 3030, 912, 1267, 1178, 1204, 527, 2019, 953] 0.0 11.826027698601226 0.0
{1, 3114, 3082, 1197, 2396, 924} [3089, 904, 3030, 912, 1267, 1178, 1204, 527, 2019, 953] 0.0 4.543559338088346 0.0
{930, 3334, 3435, 942, 2935, 912, 3095, 2203} [3089, 904, 30

{928, 1954, 34, 969, 1225, 1387, 912, 919, 3095, 1207, 2716, 541} [1148, 1218, 904, 745, 2176, 2208, 720, 750, 969, 593] 0.3010299956639812 5.092740438166795 0.059109628562248356
{3175, 3753, 112, 1968, 3512} [1148, 1218, 904, 745, 2176, 2208, 720, 750, 969, 593] 0.0 4.543559338088346 0.0
{527, 1042, 534, 151, 1179, 28, 290, 34, 1446, 1190, 41, 1203, 1208, 314, 3006, 194, 3783, 969, 3788, 1230, 1617, 1873, 2262, 346, 2908, 991, 608, 232, 2155, 1019} [1148, 1218, 904, 745, 2176, 2208, 720, 750, 969, 593] 0.3010299956639812 9.161581041840885 0.03285786528429744
{3623, 3753, 2858, 110, 3793, 3155, 3219, 86} [922, 912, 1196, 260, 2762, 527, 2324, 1278, 3030, 223] 0.0 4.543559338088346 0.0
{3608, 1193, 3326, 3174} [922, 912, 1196, 260, 2762, 527, 2324, 1278, 3030, 223] 0.0 4.543559338088346 0.0
{928, 1954, 34, 969, 1225, 1387, 912, 919, 3095, 1207, 2716, 541} [922, 912, 1196, 260, 2762, 527, 2324, 1278, 3030, 223] 0.6309297535714575 5.092740438166795 0.12388806404564567
{3175, 3753, 112, 19

{377, 2858, 1484, 1589} [922, 2571, 2019, 260, 720, 1148, 318, 858, 2762, 2858] 0.2890648263178879 4.543559338088346 0.06362078819895171
{1, 1265, 2671} [922, 2571, 2019, 260, 720, 1148, 318, 858, 2762, 2858] 0.0 4.543559338088346 0.0
{1218, 2115, 1573, 1287, 2571, 3698, 1275} [922, 2571, 2019, 260, 720, 1148, 318, 858, 2762, 2858] 0.6309297535714575 4.543559338088346 0.13886244387355454
{1097, 589, 3576, 1210, 1019, 1214} [922, 2571, 2019, 260, 720, 1148, 318, 858, 2762, 2858] 0.0 4.543559338088346 0.0
{1, 2312, 3081, 16, 1682, 2067, 534, 24, 1945, 2716, 34, 1188, 1956, 934, 39, 1193, 3504, 3252, 3386, 2243, 2501, 838, 1231, 2771, 356, 2020, 2791, 1259, 235, 2797, 2028, 1263, 1648, 1271, 1784, 1916} [922, 2571, 260, 2019, 720, 318, 858, 2762, 1148, 745] 0.0 10.336726682761885 0.0
{377, 2858, 1484, 1589} [922, 2571, 260, 2019, 720, 318, 858, 2762, 1148, 745] 0.0 4.543559338088346 0.0
{1, 1265, 2671} [922, 2571, 260, 2019, 720, 318, 858, 2762, 1148, 745] 0.0 4.543559338088346 0.0
{1218,

{1380, 2054, 3175, 3882, 2699, 1676, 11, 2028, 2005, 2422, 440, 441, 3484} [318, 3114, 904, 2762, 858, 50, 527, 2329, 2028, 260] 0.3010299956639812 5.355389973203989 0.056210658265822395
{2080, 1025, 2085, 2089, 3499, 783, 2096, 595, 2102, 1022, 3034, 1405, 2142} [2762, 2329, 2028, 50, 527, 318, 593, 2396, 914, 2571] 0.0 5.355389973203989 0.0
{1344, 1219, 1284, 36, 2918, 1704, 969, 3095, 1240} [2762, 2329, 2028, 50, 527, 318, 593, 2396, 914, 2571] 0.0 4.543559338088346 0.0
{2688, 2763, 2396, 2581} [2762, 2329, 2028, 50, 527, 318, 593, 2396, 914, 2571] 0.31546487678572877 4.543559338088346 0.06943122193677727
{1921, 2, 2692, 2054, 10, 2571, 783, 2580, 2717, 671, 32, 161, 163, 1061, 3623, 296, 2094, 1584, 1968, 1200, 2867, 1717, 2105, 1722, 317, 2622, 1342, 2243, 2372, 1732, 198, 455, 1735, 2003, 3157, 474, 2011, 733, 608, 357, 2407, 2279, 3688, 2151, 362, 110, 239, 2671, 497, 1394, 366, 2423, 1912, 2042, 380, 1021} [2762, 2329, 2028, 50, 527, 318, 593, 2396, 914, 2571] 0.289064826317887

{0: {'ADD': {'NDCG': {'mean': 0.05503926509917165,
    'min': 0.03168026121966796}},
  'MUL': {'NDCG': {'mean': 0.05503926509917165, 'min': 0.03168026121966796}},
  'LMS': {'NDCG': {'mean': 0.036779644289658804, 'min': 0.0}},
  'MPL': {'NDCG': {'mean': 0.07597877877899648, 'min': 0.07355928857931761}}},
 1: {'ADD': {'NDCG': {'mean': 0.0316088333445714, 'min': 0.0}},
  'MUL': {'NDCG': {'mean': 0.0316088333445714, 'min': 0.0}},
  'LMS': {'NDCG': {'mean': 0.05775210351419438, 'min': 0.0}},
  'MPL': {'NDCG': {'mean': 0.0, 'min': 0.0}}},
 2: {'ADD': {'NDCG': {'mean': 0.0, 'min': 0.0}},
  'MUL': {'NDCG': {'mean': 0.033127111727194514, 'min': 0.0}},
  'LMS': {'NDCG': {'mean': 0.07811904014296017, 'min': 0.016619934732171264}},
  'MPL': {'NDCG': {'mean': 0.02461404202686055, 'min': 0.0}}},
 3: {'ADD': {'NDCG': {'mean': 0.09174188877385993,
    'min': 0.051358492238643604}},
  'MUL': {'NDCG': {'mean': 0.07815911103915014, 'min': 0.05382121602276955}},
  'LMS': {'NDCG': {'mean': 0.12975370392553